In [12]:
def backend_select(backend='colab'):
  sol_path = '/home/sdianaty/pdm'
  colab_path = '/content/drive/MyDrive/SHARES'
  if backend=='colab':
    return colab_path
  elif backend=='sol':
    return sol_path
  else:
    raise ValueError("backend must be one of 'sol' or 'colab'")
    # plan to add local option

In [13]:
import os
from pathlib import Path
from funcs.requirements_utils import install_missing

backend = 'colab' # Switch backend

if backend == 'colab':
  from google.colab import drive
  drive.mount('/content/drive', force_remount=True)

PROJECT_DIR = backend_select(backend)
os.chdir(PROJECT_DIR)
print("Current Working Directory:", Path.cwd())

installed = install_missing(Path(PROJECT_DIR) / "requirements.txt", quiet=False)
print("Installed:", installed)

Mounted at /content/drive
Current Working Directory: /content/drive/MyDrive/SHARES
Installed: []


In [14]:
# Load config
from funcs.config_loader import load_config, validate_config

CONFIG_PATH = os.path.join(PROJECT_DIR, "config.json")
cfg = validate_config(load_config(CONFIG_PATH))
cfg

{'dataset': './input/dataset.xlsx',
 'output_path': './output',
 'patient_col': 'Patient',
 'physician_col': 'Physician',
 'item_col': 'Item',
 'classes': ['Behavioral health',
  'Diagnoses',
  'Disabilities',
  'Infectious diseases',
  'Genetics',
  'Medications',
  'Sexual and reproductive health',
  'Social determinants of health',
  'Violence',
  'Other'],
 'other_class': ['Disabilities',
  'Infectious diseases',
  'Genetics',
  'Sexual and reproductive health',
  'Violence',
  'Social determinants of health'],
 'llms': ['sentence-transformers/all-MiniLM-L6-v2',
  'sentence-transformers/all-mpnet-base-v2',
  'sentence-transformers/multi-qa-MiniLM-L6-cos-v1',
  'pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb',
  'sentence-transformers/biomed-mpnet-base',
  'cambridgeltl/SapBERT-from-PubMedBERT-fulltext',
  'bert-base-uncased',
  'roberta-base',
  'distilbert-base-uncased',
  'nreimers/MiniLM-L6-H384-uncased',
  'emilyalsentzer/Bio_ClinicalBERT',
  'monologg/biobert_v1.1_pub

In [15]:
# Create run directory + logger, then run pipeline
from funcs.paths import make_run_dir
from funcs.logger_setup import setup_logger
from funcs.preprocessing import run
from pathlib import Path

run_dir = make_run_dir(cfg["output_path"])
logger = setup_logger(run_dir / "log.txt")

# IMPORTANT: pipeline writes outputs into cfg["output_path"].
# We'll point cfg output_path at this run directory for this run.
#note: resolve later
cfg_run = dict(cfg)
cfg_run["output_path"] = str(run_dir)

[18:56:22 02-01-26] INFO: Logging initialized at: output/18-56_FEB-01/log.txt


In [16]:
p = run(cfg_run)
train = p['paths']['train_csv']
test = p['paths']['test_csv']
train_map = p['paths']['train_map']
test_map = p['paths']['test_map']

FileNotFoundError: Dataset file not found at: input/dataset.xlsx